### Retrieve the unemployment rate of Connecticut from 2012 through 2016

In [7]:
import requests
import json
import prettytable
import pandas as pd
from collections import defaultdict

In [8]:
economic_area_to_county= {
    'Bridgeport-Stamford-Norwalk':['Fairfield'],
    'Waterbury':['Litchfield'],
    'New Haven':['New Haven','Middlesex'],
    'Hartford-West Hartford-East Hartford':['Hartford','Tolland'],
    'Norwich-New London-Westerly':['New London'],
    'Worcester':['Windham']
}

econmic_area_series_id ={
    'Bridgeport-Stamford-Norwalk':['LAUMT097195000000003','LAUMT097195000000004','LAUMT097195000000005','LAUMT097195000000006'],
    'Waterbury':['LAUMT097870000000003','LAUMT097870000000004','LAUMT097870000000005','LAUMT097870000000006'],
    'New Haven':['LAUMT097570000000003','LAUMT097570000000004','LAUMT097570000000005','LAUMT097570000000006'],
    'Hartford-West Hartford-East Hartford':['LAUMT097345000000003','LAUMT097345000000004','LAUMT097345000000005','LAUMT097345000000006'],
    'Norwich-New London-Westerly':['LAUMT097645000000003','LAUMT097645000000004','LAUMT097645000000005','LAUMT097645000000006'],
    'Worcester':['LAUMT257960000000003','LAUMT257960000000004','LAUMT257960000000005','LAUMT257960000000006']
}

periods = {'M12':'Dec','M11':'Nov','M10':'Oct','M09':'Sept','M08':'Aug','M07':'July','M06':'Jun','M05':'May',
         'M04':'Apr','M03':'Mar','M02':'Feb','M01':'Jan'}

In [9]:
county_to_economic_area = {c: ea for ea, county in economic_area_to_county.items() for c in county}
county_to_series_id = {county: econmic_area_series_id.get(ea) for county, ea in county_to_economic_area.items()}
# county_to_series_id

In [10]:
unemployment_df = pd.DataFrame()
unemployment_df['County'] = ''
unemployment_df['Year'] =''
unemployment_df['Period'] =''
unemployment_df['Unemployment Rate'] =''
unemployment_df['Footnote'] =''

In [11]:
unemployment_df

,County,Year,Period,Unemployment Rate,Footnote


### Using BLS API to retrieve the data for 8 counties in Connecticut

In [12]:

headers = {'Content-type': 'application/json'}
start_year, end_year = '2012', '2017'
index = 0
for county, sids in county_to_series_id.items():
    print(county)
#     print(sids)
    data = json.dumps({"seriesid": sids,"startyear":start_year, "endyear":end_year})

    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)

    json_data = json.loads(p.text)
    
    for series in json_data['Results']['series']:
        x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
        seriesId = series['seriesID']

        for item in series['data']:
            year = item['year']
            period = item['period']
            value = item['value']
            footnotes=""
            for footnote in item['footnotes']:
                if footnote:
                    footnotes = footnotes + footnote['text'] + ','
            
            if '.' in value:
                unemployment_df.loc[index, 'County'] = county
                unemployment_df.loc[index, 'Year'] = year
                unemployment_df.loc[index, 'Period'] = periods.get(period)
                unemployment_df.loc[index, 'Unemployment Rate'] = value
                unemployment_df.loc[index, 'Footnote'] = footnotes[0:-1]
            index +=1

Fairfield
Litchfield
New Haven
Middlesex
Hartford
Tolland
New London
Windham


In [18]:
# Visualize the dataframe
unemployment_df.head()

,County,Year,Period,Unemployment Rate,Footnote
0,Fairfield,2017,Dec,3.9,"Data were subject to revision on April 20, 2018."
1,Fairfield,2017,Nov,4.1,"Data were subject to revision on April 20, 2018."
2,Fairfield,2017,Oct,4.2,"Data were subject to revision on April 20, 2018."
3,Fairfield,2017,Sept,4.2,"Data were subject to revision on April 20, 2018."
4,Fairfield,2017,Aug,4.6,"Data were subject to revision on April 20, 2018."


In [23]:
# Save to a csv file
unemployment_df.to_csv('Resources/Unemployment_rate_by_county.csv', index=False)